# The problem

There are 100 fresh plums on a plum tree including 3 rotten ones. Every day, each plum has a 35% chance of rotting, if they are over 2 days old. Every day, 3 fresh plums grow. The farmer comes and takes 75% of the rotten plums every 4 days, stopping the rot by 2 days. After 2 weeks:
1. How many fresh plums are there?
2. How many rotten plums are still on the tree?
3. How many total plums are there?
4. How many rotten plums did the farmer collect?
5. How many rotten plums were there in total?

# The representations

## Observable Variables


Symbol | Notes
:---:|:------
$D$ | Day
$P$ | Total plums
$R$ | Number of rotten plums
$G$ | Number of good plums



## Hidden Variables and Parameters

Consider Day 0 is equivlent to a farmer's picking, i.e. there will be no infection on Day 1 and 2.

Symbol | Notes
:---:|:------
$M$ | Number of plums subjected to the rotting diseases, i.e. good plums > 2 days
$N$ | Temporary variable ($N=R + 0.35 * M * f_1$)
$f_1$ | Farmer factor-1,to activate the rotting infection, $f_1 =
\left\{
    \begin{array}{lll}
        0  & \quad \mbox{for day 1 & 2 after farmer's dis-infection day} \\
        1  & \quad \mbox{for other days} 
    \end{array}
\right.$
$f_2 $ | Farmer factor-2, to remove rotten plums, $f_2 =
\left\{
    \begin{array}{ll}
        0.75  & \quad \mbox{on Farmer's dis-infection day} \\
        0     & \quad \mbox{else where} 
    \end{array}
\right.$


# Equations

Let $k$ be the day index
\begin{align}
P^{k+1} & = P^k + 3 - f_2 * N \\
R^{k+1} & = (1 -f_2) * N \\
G^{k+1} & = P^k + 3 - N \\
-- &- -------- \\
M & = G^k -6 \\
N & = R^k + 0.35 * M * f_1\\
\\
\end{align}
Note, previously, the 3rd equation is $G^{k+1} = P^{k+1} - R^{k+1}$. Combining eq[1] and eq[2] together, we have 

\begin{eqnarray}
\\
G^{k+1} &= & P^{k+1} - R^{k+1} \\
        &= & (P^k + 3 - f_2 * N) - (1-f_2) * N \\
        &= & P^k + 3 - f_2 * N - N + f_2 * N \\
        &= & P^k + 3 - N
\end{eqnarray}

Hence, the new eq[3]

# Solve the equations

From here, there are two approaches to solve those equations -- brutal force, or NN (which offers something intelligent)

## Brutal force

We treat $M$ and $N$ as $M^{k+1}$ and $N^{k+1}$ for the $k+1$ day

\begin{align}
P^{k+1} + f_2 * N^{k+1} &= P^k + 3 \\
R^{k+1} - (1-f_2)*N^{k+1} &= 0 \\
G^{k+1} + N^{k+1} &= P^k + 3 \\
------- &- --- \\
M^{k+1} &= G^k - 6 \\
-0.35 * f_1 * M^{k+1} + N^{k+1} &= R^k \\
\end{align}

Let $\mathbf{X}$ be 
$$\mathbf{X} = \begin{bmatrix}
P\\
R \\
G \\
--\\
M \\
N \\
\end{bmatrix}
$$


\begin{equation}
\begin{bmatrix} 
1 & 0 & 0 & 0 & f_2 \\
0 & 1 & 0 & 0 & -(1-f_2)\\
0 & 0 & 1 & 0 & 1 \\
0 & 0 & 0 & 1 & 0 \\
0 & 0 & 0 & -0.35 * f_1 & 1
\end{bmatrix} 
\begin{bmatrix}
P\\
R \\
G \\
M \\
N
\end{bmatrix}^{k+1} = 
\begin{bmatrix} 
1 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 \\
1 & 0 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 & 0 \\
0 & 1 & 0 & 0 & 0
\end{bmatrix}
\begin{bmatrix}
P\\
R \\
G \\
M \\
N
\end{bmatrix}^k
+
\left[
\begin{array}{r}
3\\
0 \\
3 \\
-6\\
0
\end{array}
\right]
\end{equation}

We then have 
\begin{equation}
W_1 \mathbf{X}^{k+1} = W_2 \mathbf{X}^k + B
\end{equation}

where
\begin{equation}
\begin{array}{}
W_1 = \begin{bmatrix} 
1 & 0 & 0 & 0 & f_2 \\
0 & 1 & 0 & 0 & -(1-f_2) \\
0 & 0 & 1 & 0 & 1 \\
0 & 0 & 0 & 1 & 0 \\
0 & 0 & 0 & -0.35*f_1 & 1
\end{bmatrix} 
,&
W_2 = \begin{bmatrix} 
1 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 \\
1 & 0 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 & 0 \\
0 & 1 & 0 & 0 & 0
\end{bmatrix},
&
B = \left[
\begin{array}{r}
3 \\
0 \\
3 \\
-6 \\
0
\end{array}
\right]
\end{array}
\end{equation}

In [1]:
import numpy as np
import scipy as sp


## Define a function to do the calculation

To calculate today's Y, based on parameters f1 and f2, and the values from the previous day.

In [2]:
def dx_cal(X, f1, f2) :
    W1 = np.array([[1,  0, 0, 0, f2],
                   [0,  1, 0, 0, -(1-f2)],
                   [0,  0, 1, 0, 1],
                   [0,  0, 0, 1, 0],
                   [0,  0, 0, -0.35*f1, 1]])

    W2 = np.array([[1, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0],
                   [1, 0, 0, 0, 0],
                   [0, 0, 1, 0, 0],
                   [0, 1, 0, 0, 0]])

    B = np.array([[ 3, 0, 3,-6, 0]]).T
    
    W1_inv = np.linalg.inv(W1)
    
    Y = W1_inv.dot(W2.dot(X) + B)
    return np.rint(Y)


Now, I noticed that the crucial this is the values of f-factors. For now, I can create a lookup table


In [3]:
F = np.array([[0, 0],      # for Day 1
              [0, 0],      # Day2
              [1, 0],      # Day3
              [1, 0.75],   # Day4
              [0, 0],      # Day5
              [0, 0],
              [1, 0],
              [1, 0.75],
              [0, 0],
              [0, 0],
              [1, 0],
              [1, 0.75],
              [0, 0],
              [0, 0],
              [1, 0],
              [1, 0.75]])
Results = np.array([[100, 3, 97, 0, 0 ]])    # Day0 values, the initial value, M and N doesn't matter, so give them 0
for k in range(14):
    X_k = np.array([Results[k]]).T
    X_k1 = dx_cal(X_k, F[k,0], F[k,1])
    Results = np.concatenate((Results, X_k1.T), axis=0)

In [4]:
Results

array([[ 100.,    3.,   97.,    0.,    0.],
       [ 103.,    3.,  100.,   91.,    3.],
       [ 106.,    3.,  103.,   94.,    3.],
       [ 109.,   37.,   72.,   97.,   37.],
       [  67.,   15.,   52.,   66.,   60.],
       [  70.,   15.,   55.,   46.,   15.],
       [  73.,   15.,   58.,   49.,   15.],
       [  76.,   33.,   43.,   52.,   33.],
       [  45.,   11.,   33.,   37.,   46.],
       [  48.,   11.,   37.,   27.,   11.],
       [  51.,   11.,   40.,   31.,   11.],
       [  54.,   23.,   31.,   34.,   23.],
       [  33.,    8.,   25.,   25.,   32.],
       [  36.,    8.,   28.,   19.,    8.],
       [  39.,    8.,   31.,   22.,    8.]])